#### Setting up praw which allows us to work with reddit


In [1]:
import  praw
import pandas as pd

my_client_id = 'nLqFb50p3KTbWQ'
my_secret = 'zVGm8qyGW1lcNNbu43rxoSZ8fq7K3g'
my_user_agent = 'redditanalysis0'
reddit = praw.Reddit(client_id= my_client_id, client_secret=my_secret, user_agent= my_user_agent)



#### Analysing the Skincare Addiction subreddit to understand which products people use most.
1) Collect necessary posts.



In [2]:
needed_word = '[shelfie]'

hot_posts = reddit.subreddit('SkincareAddiction').hot(limit=None)
new_posts = reddit.subreddit('SkincareAddiction').new(limit=None)
rising_posts = reddit.subreddit('SkincareAddiction').rising(limit=None)
top_posts = reddit.subreddit('SkincareAddiction').top(limit=None)
controversial_posts = reddit.subreddit('SkincareAddiction').controversial(limit=None)
gilded_posts = reddit.subreddit('SkincareAddiction').gilded(limit=None)



2) Extract all posts of people posting their shelfies.


In [3]:
shelfie_posts = []

for posts in [hot_posts,new_posts, top_posts, rising_posts,controversial_posts]:
    for post in posts:
        if needed_word in post.title.lower():
            shelfie_posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])

shelfie_posts =  pd.DataFrame(shelfie_posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
print(len(shelfie_posts))
#143



130


In [4]:
shelfie_posts.drop_duplicates(subset ="title", inplace = True)
shelfie_posts.sort_values('score', ascending=False, inplace = True)
print(shelfie_posts)



                                                 title  score      id  \
22                                Watercolor [shelfie]   7543  9vvzzx   
23                        [Shelfie] The duality of man   7465  f8qa0m   
24   [Shelfie] I’m an illustrator and drew my skinc...   7014  h9oy2y   
25   [Shelfie] My skincare drawer sparks joy this m...   6761  iz1i6m   
26   [Haul] [Shelfie] My boyfriend bought me a skin...   6667  at65xl   
..                                                 ...    ...     ...   
94   [shelfie] I was struggling with acne until my ...      0  ifn64m   
88       [SHELFIE] I think I​ need an Intervention! :P      0  axus2w   
89   [Shelfie] Too much product for a 39 year old w...      0  81kzgm   
93    [Shelfie] I realized my shelf is getting full...      0  ahilo0   
107  [shelfie] Just the essentials...I guess you co...      0  aqtgvl   

             subreddit                                                url  \
22   SkincareAddiction                https://

In [5]:
shelfie_posts.to_csv(r'shelfie_posts.csv', index = False)


3) Find most often met words. Do some NLP shit.


In [6]:
import numpy as np
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt

In [7]:
id_to_inspect = shelfie_posts.iloc[0].id
print(id_to_inspect, shelfie_posts.iloc[0].title)

submission = reddit.submission(id=id_to_inspect)
submission.comments.replace_more(limit=None)

comments = []

for top_level_comment in submission.comments.list():
    comments.append(top_level_comment.body)
comments





9vvzzx Watercolor [shelfie]


['If you correctly guessed what all of those products were just by looking at the painting you spend too much time on this sub and we are besties ',
 'That’s seriously super cool. I need that hanging in my bathroom.\n',
 'I had to do an autobiographical still life for my painting class, and thought of my skincare routine, naturally.\n\nProducts:\n\n* Vanicream moisturizer\n* Elta MD UV Clear SPF 46\n* Timeless Vitamin C\n* Curology\n\nSkincare goals:\n\nProtect and defend/have good skin at 50 years old',
 "That's the best shelfie I've seen on this sub.",
 'Is that Vanicream? Love that stuff. Such a good painting!',
 '[deleted]',
 'Oh I LOVE this. You should sell these! ',
 'Ohhhhh. I love this. ',
 'This is the first shelfie I actually want to compliment. Really well done!',
 "I'm been on this sub so long that I recognized all of the products XD",
 'Love the timeless! It’s the best. ',
 'Beautiful use of light and shadow!',
 'adore this',
 'I absolutely love this. Thank you for sharing

4.1) Collect all comments from all posts into one text



In [8]:
i = 0
comments = []

for index, post in shelfie_posts.iterrows():

    id_to_inspect = post.id

    submission = reddit.submission(id=id_to_inspect)
    submission.comments.replace_more(limit=None)

    for top_level_comment in submission.comments.list():
        comments.append(top_level_comment.body)
    i+=1



In [9]:
len(comments)


11739

In [10]:
text = " ".join(comment for comment in comments)
text

f = open("all_comments.txt", "x")
f.write(text)
f.close()



FileNotFoundError: [Errno 2] No such file or directory: 'data/all_comments.txt'